In [ ]:
# Installs
!python3 -m pip install -e .
!pip install openml
!pip install keras
!pip install -q -U keras-tuner

In [2]:
# Imports
import openml
import pandas as pd
from time import time
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import scipy.stats as stats
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from tensorflow.keras import initializers
from openml.datasets import edit_dataset, fork_dataset, get_dataset
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Used this video to help understand and use GAN
https://www.youtube.com/watch?v=yujdA46HKwA&t=82s&ab_channel=JeffHeaton

In [3]:
# HIDE OUTPUT
# CMD = "wget https://raw.githubusercontent.com/Diyago/"\
#   "GAN-for-tabular-data/master/requirements.txt"

# !{CMD}
# !pip install -r requirements.txt
# ABOVE IS NOT NEEDED ^


!pip install tabgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.3 MB/s eta 0:00:00


Auto MPG dataset to train a GAN to generate fake cars.
https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Farxiv.org%2Fpdf%2F2010.00638.pdf

In [ ]:
# Get TPU
# IGNORE if no TPU
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [4]:
# This is done based on the dataset ID.
dataset = openml.datasets.get_dataset(42890)

# Storing the data
eeg, *_ = dataset.get_data()

/usr/local/lib/python3.10/dist-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


# Todo

Create a GAN

Feed the gan a data set with only 1s
ensure that is scalled to meet the same number of rows as 0s

reproduce old note book but cleaner

In [ ]:
eeg.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [5]:
# Get failures
filtered_eeg = eeg[eeg['Machine failure'] == 1]

In [6]:
# Remove UDI as it is redundant
filtered_eeg = filtered_eeg.drop(columns=['UDI'])

In [7]:
filtered_eeg.head(1000)

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
50,L47230,L,298.9,309.1,2861,4.6,143,1,0,0,1,0,0
69,L47249,L,298.9,309.0,1410,65.7,191,1,0,0,1,1,0
77,L47257,L,298.8,308.9,1455,41.3,208,1,1,0,0,0,0
160,L47340,L,298.4,308.2,1282,60.7,216,1,0,0,0,1,0
161,L47341,L,298.3,308.1,1412,52.3,218,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,L56938,L,298.6,309.8,2271,16.2,218,1,1,0,0,0,0
9764,L56944,L,298.5,309.5,1294,66.7,12,1,0,0,1,0,0
9822,L57002,L,298.5,309.4,1360,60.9,187,1,0,0,0,1,0
9830,L57010,L,298.3,309.3,1337,56.1,206,1,0,0,0,1,0


In [9]:
# Convert all numerical columns to floats
filtered_eeg = filtered_eeg.apply(lambda col: col.astype('int') if col.dtypes == 'float64' else col)
filtered_eeg.dtypes

Product ID                 object
Type                       object
Air temperature [K]         int64
Process temperature [K]     int64
Rotational speed [rpm]      int64
Torque [Nm]                 int64
Tool wear [min]             int64
Machine failure             int64
TWF                         int64
HDF                         int64
PWF                         int64
OSF                         int64
RNF                         int64
dtype: object

In [10]:
# Again altering datatypes
filtered_eeg['Torque [Nm]'] = filtered_eeg['Torque [Nm]'].astype('float')
filtered_eeg.dtypes


Product ID                  object
Type                        object
Air temperature [K]          int64
Process temperature [K]      int64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object

In [11]:
# Rename columns as the GAN doesn't like the original column names
filtered_eeg = filtered_eeg.rename(columns={'Air temperature [K]': 'Air_temperature',
                                            'Process temperature [K]': 'Process_temperature',
                                            'Rotational speed [rpm]': 'Rotational_speed',
                                            'Torque [Nm]': 'Torque',
                                            'Tool wear [min]': 'Tool_wear'})

In [12]:
# Break up data and create the train test splits
from sklearn.model_selection import train_test_split

df_filtered = filtered_eeg.drop(columns=['Product ID', 'Type', 'Machine failure'])
target_cols = filtered_eeg.drop(columns=['Product ID', 'Type', 'Air_temperature',
                  'Process_temperature', 'Rotational_speed',
                  'Torque', 'Tool_wear', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])

X = df_filtered
y = target_cols

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# # Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

In [13]:
# Gan to create new data
from tabgan.sampler import GANGenerator
from sklearn.model_selection import train_test_split

# Building GAN and adding the data
gen_x, gen_y = GANGenerator(gen_x_times=36, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           ).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

/usr/local/lib/python3.10/dist-packages/_ctgan/synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Fitting CTGAN transformers for each column:   0%|          | 0/11 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 54, number of negative: 54
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001674 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 122
[LightGBM] [Info] Number of data points in the train set: 108, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [14]:
# convert to df
gan_x = pd.DataFrame(gen_x)
gan_y = pd.DataFrame(gen_y)


In [15]:
gan_x.shape

(10159, 10)

In [16]:
X.shape

(339, 10)

In [17]:
X_combine = pd.concat([X, gan_x], ignore_index=True)
y_combine = pd.concat([y, gan_y], ignore_index=True)

In [18]:
X_combine.shape

(10498, 10)

In [19]:
df = X_combine
df.head()

,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,TWF,HDF,PWF,OSF,RNF
0,298,309,2861,4.0,143,0,0,1,0,0
1,298,309,1410,65.0,191,0,0,1,1,0
2,298,308,1455,41.0,208,1,0,0,0,0
3,298,308,1282,60.0,216,0,0,0,1,0
4,298,308,1412,52.0,218,0,0,0,1,0


In [20]:
# Combine target and features
df.insert(0, 'Machine failure', y_combine)

In [21]:
# change the order of cols in df
df = df[[ 'Air_temperature', 'Process_temperature', 'Rotational_speed', 'Torque', 'Tool_wear', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']]

In [22]:
df.head()

,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine failure,TWF,HDF,PWF,OSF,RNF
0,298,309,2861,4.0,143,1,0,0,1,0,0
1,298,309,1410,65.0,191,1,0,0,1,1,0
2,298,308,1455,41.0,208,1,1,0,0,0,0
3,298,308,1282,60.0,216,1,0,0,0,1,0
4,298,308,1412,52.0,218,1,0,0,0,1,0


# What has happened?

So the data created by the gan has benn converted to a df, then concatenated to the X, y splits respectivly.

Then appended and ordered into a new df

In [23]:
no_fails = eeg[eeg['Machine failure'] == 0]

In [24]:
df = df.rename(columns={'Air_temperature': 'Air temperature [K]',
                                            'Process_temperature': 'Process temperature [K]',
                                            'Rotational_speed': 'Rotational speed [rpm]',
                                            'Torque': 'Torque [Nm]',
                                            'Tool_wear': 'Tool wear [min]'})

In [25]:
# All data back together
eeg_new = pd.concat([no_fails, df], ignore_index=True)

In [26]:
eeg_new = eeg_new.drop(columns=['UDI', 'Product ID', 'Type'])
eeg_new.shape

(20159, 11)

In [27]:
# Prepare for neural nets
df_X = eeg_new.drop(columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
df_y = eeg_new.drop(columns=['Air temperature [K]',
                  'Process temperature [K]', 'Rotational speed [rpm]',
                  'Torque [Nm]', 'Tool wear [min]'])

X = df_X
y = df_y

In [55]:
# Prepare for neural nets
# Don't include machine failure
df_X = eeg_new.drop(columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'])
df_y = eeg_new.drop(columns=['Air temperature [K]',
                  'Process temperature [K]', 'Rotational speed [rpm]',
                  'Torque [Nm]', 'Tool wear [min]', 'Machine failure'])

X = df_X
y = df_y

In [31]:
# Cross validiation using numpy

from sklearn.model_selection import KFold
import numpy as np

X_np = X.to_numpy()
y_np = y.to_numpy()

kf = KFold(n_splits=17, shuffle=True, random_state=62)

for train_index, test_index in kf.split(X_np):
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]

In [33]:
# Ignore if using cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)



---



In [55]:
# Define the model
tf_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(300, activation=tf.nn.sigmoid, input_shape=(X_train.shape[1],)), # Hidden layer def
    tf.keras.layers.Dropout(0.35), # To reduce overfitting
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid') # Output layer (Adjust based on data used)
])

In [56]:
tf_model.compile(keras.optimizers.Adadelta(
    learning_rate=0.0003,
    rho=0.95,
    epsilon=1e-07,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    name="adadelta"),
loss=tf.keras.losses.BinaryCrossentropy(),
metrics=[tf.keras.metrics.Recall(),
         tf.keras.metrics.FalseNegatives()])


In [57]:
tf_model.fit(X_train, y_train, epochs=20)
y_pred = tf_model.predict(X_test)

Epoch 1/20
441/441 [==============================] - 2s 3ms/step - loss: 1.2066 - recall_6: 0.7950 - false_negatives_6: 1948.0000
Epoch 2/20
441/441 [==============================] - 1s 3ms/step - loss: 1.1826 - recall_6: 0.7912 - false_negatives_6: 1984.0000
Epoch 3/20
441/441 [==============================] - 1s 3ms/step - loss: 1.1541 - recall_6: 0.7734 - false_negatives_6: 2153.0000
Epoch 4/20
441/441 [==============================] - 1s 3ms/step - loss: 1.1221 - recall_6: 0.7580 - false_negatives_6: 2299.0000
Epoch 5/20
441/441 [==============================] - 1s 3ms/step - loss: 1.0876 - recall_6: 0.7367 - false_negatives_6: 2502.0000
Epoch 6/20
441/441 [==============================] - 1s 3ms/step - loss: 1.0586 - recall_6: 0.7145 - false_negatives_6: 2713.0000
Epoch 7/20
441/441 [==============================] - 1s 3ms/step - loss: 1.0217 - recall_6: 0.7029 - false_negatives_6: 2823.0000
Epoch 8/20
441/441 [==============================] - 1s 3ms/step - loss: 0.9876 - 

# Results

It's slightly concerning that the above has such high false negatives

There is way more 1s in this train and test data then previously but I'm somehow still seeing a proportional amount of false negs....

In [58]:
def build_model(hp):

    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 3, 9)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=128, max_value=1344, step=16),
                activation=hp.Choice("activation", ["sigmoid", "relu"]),
            )
        )

    # Silu is defined as: swish(x) = x * sigmoid(x).
    # This activation function is a smooth, non-monotonic function that is unbounded above and bounded below.

    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(6, activation="sigmoid"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          optimizer=keras.optimizers.Adadelta(
              learning_rate=0.0001,
              rho=0.95, # Decay rate
              epsilon=1e-07,
              weight_decay=None,
              clipnorm=None,
              clipvalue=None,
              global_clipnorm=None,
              use_ema=False,
              ema_momentum=0.91,
              ema_overwrite_frequency=None,
              name="adadelta"),
          metrics=[
                # tf.keras.metrics.TruePositives(name='tp'),
                # tf.keras.metrics.FalsePositives(name='fp'),
                # tf.keras.metrics.TrueNegatives(name='tn'),
                # tf.keras.metrics.FalseNegatives(name='fn'),
                # tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                # tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                #tf.keras.metrics.AUC(name='auc'),
    ]),
    return model


build_model(kt.HyperParameters())

In [64]:
# Using Hyperband
hyper2_tuner = kt.Hyperband(
    hypermodel=build_model,
    objective=kt.Objective('recall', direction='max'),
    max_epochs=20,
    factor=3,
    hyperband_iterations=3,
    max_retries_per_trial=0,
    overwrite=True,
    directory="/",
    project_name="NewProject",
    max_consecutive_failed_trials=3
)

In [61]:
hyper2_tuner.search(X_train, y_train, epochs=8, validation_data=(X_test, y_test), verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

Trial 90 Complete [00h 01m 24s]
recall: 0.3239659070968628

Best recall So Far: 1.0
Total elapsed time: 00h 24m 54s


In [ ]:
hyper2_tuner.search(X_train, y_train, epochs=8, validation_data=(X_test, y_test), verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

Trial 40 Complete [00h 00m 10s]
recall: 0.8893800377845764

Best recall So Far: 0.8893800377845764
Total elapsed time: 00h 10m 12s

Search: Running Trial #41

Value             |Best Value So Far |Hyperparameter
8                 |7                 |num_layers
224               |288               |units_0
sigmoid           |sigmoid           |activation
560               |1120              |units_1
496               |1168              |units_2
True              |False             |dropout
0.00021802        |0.00070118        |lr
688               |768               |units_3
1264              |400               |units_4
272               |1232              |units_5
336               |1120              |units_6
176               |336               |units_7
496               |640               |units_8
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0        